# Tohoku Okada Notebook

## Setup Environment

We assume you have installed `anuga` in your `python` environment. 

### Setup inline graphics and animation

In [ ]:
import os
import anuga
import numpy as np
import matplotlib.pyplot as plt
#from google.colab import files

%matplotlib inline

# Allow inline jshtml animations
from matplotlib import rc
rc('animation', html='jshtml')

## Tohoku Folder

Change into the Tohoku folder (if you have not done that already)

In [ ]:
print(f'Current directory {os.getcwd()}')
try:
    os.chdir('Tohoku')
except:
    pass


## Setup Anuga Model

### Project parameters

In [ ]:
import project


In [ ]:
print ('project name: ', project.name_stem)
domain = anuga.create_domain_from_regions(project.bounding_polygon,
                                        boundary_tags={'bottom': [0],
                                                       'ocean_east': [1],
                                                       'top': [2],
                                                       'onshore': [3]},
                                        maximum_triangle_area=project.res_whole,
                                        mesh_filename=project.meshname,
                                        interior_regions=project.interior_regions,
                                        use_cache=False,
                                        verbose=False)



domain.set_hemisphere('northern')
domain.set_zone(54)
print ('Number of triangles = ', len(domain))
print ('The extent is ', domain.get_extent())
print ('Geo reference is ',domain.geo_reference)

domain.set_quantity('elevation',
                        filename=project.name_stem + '.pts',
                        use_cache=True,
                        verbose=False,
                        alpha=0.1)


tide = -0.45
domain.set_quantity('stage', tide)


Elevation = domain.quantities['elevation'].centroid_values
Stage     = domain.quantities['stage'].centroid_values

Stage[:] = np.maximum(Elevation, Stage)
                        

dplotter = anuga.Domain_plotter(domain, absolute=True)
plt.triplot(dplotter.triang, linewidth = 0.4);
#print (domain.statistics())

#domain.set_quantity('elevation', function=topography, location='centroids')

### Setup fault properties

In [ ]:
#from anuga.tsunami_source.okada_tsunami import earthquake_source

# location of fault (relative coordinates)
x0 = 300000.0
y0 = 250000.0

depth = 15000.0
strike = 0.0
dip = 15.0
length = 20000.0
width = 6000.0
slip = 60.0
rake = 90.0
opening = 0.0
nu = 0.25

In [ ]:
import cartopy
import cartopy.crs as ccrs

fig = plt.figure()
ax = fig.add_subplot(111, projection=ccrs.UTM(zone=54))
#ax = fig.add_subplot(111)

#ax.coastlines()

xll = domain.geo_reference.xllcorner
yll = domain.geo_reference.yllcorner


md = 0.01

# Dry areas
dplotter.triang.set_mask(dplotter.depth > md)
ax.tripcolor(dplotter.triang,
                      facecolors=dplotter.elev,
                      cmap='BrBG')
                      #cmap='viridis_r')
                      #cmap='Greys_r')

# Wet Areas
dplotter.triang.set_mask(dplotter.depth <= md)
ax.tripcolor(dplotter.triang,
                      facecolors=dplotter.elev,
                      cmap='Blues_r')
                      #cmap='viridis')
                      #vmin=vmin, vmax=vmax)

dplotter.triang.set_mask(None)

ax.scatter(x0+xll, y0+yll,marker='+', color='r')



ax.set_extent([450000,900000,4000000, 4400000], crs=ccrs.UTM(zone=54))
gridlines = ax.gridlines(draw_labels=True)

#ax.coastlines()

plt.title('Elevation (Bathymetry) east coast Japan')

#plt.colorbar()



### Calculate Okada deformation and  

Calculate surface deformation based on a simple rectangular fault

In [ ]:
import okada

x = domain.centroid_coordinates[:,0]
y = domain.centroid_coordinates[:,1]

uE,uN,uZ = okada.forward(x,y, xoff=x0, yoff=y0, 
                   depth=depth, length=length, width=width,
                   slip=slip, opening=opening,
                   strike=strike, dip=dip, rake=rake,
                   nu=nu)


# Plot the deformation
dplotter = anuga.Domain_plotter(domain, absolute=True)

fig = plt.figure()
ax = fig.add_subplot(111, projection=ccrs.UTM(zone=54))
#ax = fig.add_subplot(111)

md = 0.01
#dplotter.triang.set_mask(None)

# dry areas
#dplotter.triang.set_mask(dplotter.depth > md)
img = ax.tripcolor(dplotter.triang,
                      facecolors=uZ,
                      #cmap='BrBG')
                      cmap='viridis')
                      #cmap='Greys_r')

ax.set_extent([450000,900000,4000000, 4400000], crs=ccrs.UTM(zone=54))
gridlines = ax.gridlines(draw_labels=True)

plt.colorbar(img, pad = 0.2, shrink=0.8)
ax.coastlines()

plt.title('Surface Deformation');



### Apply deformation to Stage and Elevation

In [ ]:
Elevation = domain.quantities['elevation'].centroid_values
Stage     = domain.quantities['stage'].centroid_values

Elevation[:] = Elevation + uZ
Stage[:]     = Stage + uZ


print ('Stage extent     ',np.max(dplotter.stage), np.min(dplotter.stage))
print ('Elevation extent ',np.max(dplotter.elev), np.min(dplotter.elev))
print ('Depth extent     ',np.max(dplotter.depth), np.min(dplotter.depth))

# plt the updated elevation and stage
dplotter = anuga.Domain_plotter(domain, absolute=True)

fig = plt.figure()
ax = fig.add_subplot(111, projection=ccrs.UTM(zone=54))
#ax = fig.add_subplot(111)

md = 0.01

# dry areas
dplotter.triang.set_mask(dplotter.depth > md)
ax.tripcolor(dplotter.triang,
                      facecolors=dplotter.elev,
                      cmap='BrBG')
                      #cmap='viridis_r')
                      #cmap='Greys_r')


# wet areas
dplotter.triang.set_mask(dplotter.depth <= md)
img = ax.tripcolor(dplotter.triang,
                      facecolors=dplotter.stage,
                      #cmap='Blues_r')
                      cmap='viridis')
                      #vmin=vmin, vmax=vmax)

ax.set_aspect('equal', adjustable='box')

plt.colorbar(img, pad = 0.2, shrink=0.8)

ax.scatter(300000.0+xll,250000.0+yll,marker='+', color='r')

ax.set_extent([450000,900000,4000000, 4400000], crs=ccrs.UTM(zone=54))
gridlines = ax.gridlines(draw_labels=True)



ax.coastlines()

plt.title('Stage (with dry elevation)');

### Setup Boundary Conditions

In [ ]:
Br = anuga.Reflective_boundary(domain)
Bf = anuga.Flather_external_stage_zero_velocity_boundary(domain,lambda t :tide)
# Boundary conditions
domain.set_boundary({'onshore': Br,
                         'bottom': Bf,
                         'ocean_east': Bf,
                         'top': Bf})

### Evolve the simulation

In [ ]:
import time
t0 = time.time()
min = 60
hour = 3600

dplotter = anuga.Domain_plotter(domain, absolute=True)

# Initial run without any event
for t in domain.evolve(yieldstep=5*min, finaltime=2*hour):
    dplotter.save_stage_frame(vmin=-2.0,vmax=4.0)

    domain.print_timestepping_statistics()

print ('That took %.2f seconds' %(time.time()-t0))


In [ ]:
dplotter.make_stage_animation()